In [1]:
import pandas as pd
import warnings
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000
warnings.filterwarnings('ignore')

#### Import Shipped Order Lines

In [2]:
df_lines = pd.read_csv('order_lines.csv', index_col = 0)
print("{:,} order lines to process".format(len(df_lines)))
df_lines.head()

5,208 order lines to process


,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros
112,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47
14485,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03
14486,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45
14487,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45
14488,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16


#### Import Master Data: Unit of Measure Conversions to kg

In [3]:
df_uom = pd.read_csv('uom_conversions.csv', index_col = 0)
print("{:,} Unit of Measure Conversions".format(len(df_uom)))

# join
df_join = df_lines.copy()
COLS_JOIN = ['Item Code']
df_join = pd.merge(df_join, df_uom, on = COLS_JOIN, how = 'left', suffixes=('', '_y'))
df_join.drop(df_join.filter(regex = '_y$').columns.tolist(), axis = 1, inplace = True)
print("{:,} records".format(len(df_join)))
df_join.head()

557 Unit of Measure Conversions
5,208 records


,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros,Conversion Ratio
0,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47,56.58400
1,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03,0.03200
2,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45,0.02219
3,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45,0.02219
4,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16,0.03200


#### Import Distances

In [4]:
df_dist = pd.read_csv('distances.csv', index_col = 0)
df_dist['Location'] = '-'
df_dist['Location'] = df_dist['Customer Country'].astype(str) + ', ' + df_dist['Customer City'].astype(str)
df_dist.head()

,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Road,Rail,Sea,Air,Location
19,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,765.728,0.0,0.0,0.0,"FRANCE, LES ANGLES"
610,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE"
676,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,856.000,0.0,0.0,0.0,"GERMANY, PEINE - WOLTORF"
682,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,52.694,0.0,0.0,0.0,"FRANCE, BELLEVILLE"
1375,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5029913-13,FRANCE,MOISSY-CRAMAYEL,288.437,0.0,0.0,0.0,"FRANCE, MOISSY-CRAMAYEL"


#### Import Cities GPS locations

In [5]:
df_gps = pd.read_csv('gps_locations.csv', index_col = 0)
print("{:,} Locations".format(len(df_gps)))
df_gps.head()

19 Locations


,Location,GPS 1,GPS 2
867,"BULGARIA, DOLNI BOGROV",42.701462,23.490811
1087,"FRANCE, AUBROMETZ",50.303749,2.176058
1153,"FRANCE, BELLEVILLE",48.871184,2.386682
1371,"FRANCE, CLICHY CEDEX",48.904099,2.304123
1534,"FRANCE, GAEL",48.116682,-2.234640


### Data Processing

#### Merge Distance with GPS Location

In [6]:
df_dist = pd.merge(df_dist, df_gps, on = 'Location', how = 'left', suffixes = ('', '_y'))
df_dist.drop(df_dist.filter(regex = '_y$').columns.tolist(), axis = 1, inplace = True)
df_dist

,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Road,Rail,Sea,Air,Location,GPS 1,GPS 2
0,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,765.728,0.0,0.0,0.0,"FRANCE, LES ANGLES",42.579678,2.048950
1,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
2,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,856.000,0.0,0.0,0.0,"GERMANY, PEINE - WOLTORF",52.304594,10.312521
3,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,52.694,0.0,0.0,0.0,"FRANCE, BELLEVILLE",48.871184,2.386682
4,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5029913-13,FRANCE,MOISSY-CRAMAYEL,288.437,0.0,0.0,0.0,"FRANCE, MOISSY-CRAMAYEL",48.624666,2.595756
5,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5003451-51,FRANCE,LEVALLOIS PERRET,295.165,0.0,0.0,0.0,"FRANCE, LEVALLOIS PERRET",48.894640,2.287419
6,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5293132-32,FRANCE,HERLIN LE SEC,353.590,0.0,0.0,0.0,"FRANCE, HERLIN LE SEC",50.354429,2.331766
7,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5003269-69,FRANCE,MACON,478.723,0.0,0.0,0.0,"FRANCE, MACON",48.878552,2.280117
8,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5003474-74,FRANCE,ORLY,287.110,0.0,0.0,0.0,"FRANCE, ORLY",48.740227,2.402965
9,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5003476-76,FRANCE,VITRY SUR SEINE,284.851,0.0,0.0,0.0,"FRANCE, VITRY SUR SEINE",48.789263,2.395103


#### Final Join to build records

In [7]:
COLS_JOIN = ['Warehouse Code', 'Customer Code']
df_join = pd.merge(df_join, df_dist, on = COLS_JOIN, how = 'left', suffixes = ('', '_y'))
df_join.drop(df_join.filter(regex='_y$').columns.tolist(), axis = 1, inplace = True)
print("{:,} records".format(len(df_join)))
df_join

5,208 records


,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros,Conversion Ratio,Warehouse Name,Warehouse Country,Warehouse City,Customer Country,Customer City,Road,Rail,Sea,Air,Location,GPS 1,GPS 2
0,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47,56.58400,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,LES ANGLES,765.728,0.0,0.0,0.0,"FRANCE, LES ANGLES",42.579678,2.048950
1,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03,0.03200,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
2,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45,0.02219,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
3,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45,0.02219,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
4,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16,0.03200,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5203,2021-10-28 00:00:00.000,10-2021,3403434,5397843-43,21089497,84,802065-65,80.0,72.96,0.02200,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
5204,2021-10-28 00:00:00.000,10-2021,3403434,5397843-43,21089497,85,942082-82,200.0,391.88,0.03200,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
5205,2021-10-28 00:00:00.000,10-2021,3403434,5397843-43,21089497,86,8022922-22,400.0,581.40,0.03200,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
5206,2021-11-01 00:00:00.000,10-2021,3403434,5340669-69,21004473,1,982884-84,2000.0,2980.00,0.14600,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,GERMANY,PEINE - WOLTORF,856.000,0.0,0.0,0.0,"GERMANY, PEINE - WOLTORF",52.304594,10.312521


#### Calculation at order level

In [8]:
# Calculate Weight (KG)
df_join['KG'] = df_join['Units'] * df_join['Conversion Ratio']

# Agg by order
GPBY_ORDER = ['Date', 'Month-Year', 
        'Warehouse Code', 'Warehouse Name', 'Warehouse Country', 'Warehouse City',
        'Customer Code', 'Customer Country', 'Customer City','Location', 'GPS 1', 'GPS 2', 
        'Road', 'Rail', 'Sea', 'Air',
        'Order Number', 'Item Code']
df_agg = pd.DataFrame(df_join.groupby(GPBY_ORDER)[['Euros', 'Units', 'KG']].sum())
df_agg.reset_index(inplace = True)
df_agg.head()

,Date,Month-Year,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Location,GPS 1,GPS 2,Road,Rail,Sea,Air,Order Number,Item Code,Euros,Units,KG
0,2021-01-04 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,"FRANCE, LES ANGLES",42.579678,2.048950,765.728,0.0,0.0,0.0,20247100,963543-43,381.47,2.0,113.16800
1,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,"GERMANY, PEINE - WOLTORF",52.304594,10.312521,856.000,0.0,0.0,0.0,20203388,993081-81,24374.43,153.0,2530.32471
2,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,"FRANCE, BELLEVILLE",48.871184,2.386682,52.694,0.0,0.0,0.0,20203383,993081-81,20391.68,128.0,2116.87296
3,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,802077-77,408.27,300.0,5.70000
4,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,8022921-21,567.16,400.0,12.80000


#### Calculate CO2 = f(KG, Ratios)

In [9]:
# CO2 Emissions
dict_co2e = dict(zip(['Air' ,'Sea', 'Road', 'Rail'], [2.1, 0.01, 0.096, 0.028]))
MODES = ['Road', 'Rail','Sea', 'Air']
for mode in MODES:
    df_agg['CO2 ' + mode] = df_agg['KG'].astype(float)/1000 * df_agg[mode].astype(float) * dict_co2e[mode]
df_agg['CO2 Total'] = df_agg[['CO2 ' + mode for mode in MODES]].sum(axis = 1)
df_agg.to_csv('detailed_report.csv')
df_agg.head()

,Date,Month-Year,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Location,GPS 1,GPS 2,Road,Rail,Sea,Air,Order Number,Item Code,Euros,Units,KG,CO2 Road,CO2 Rail,CO2 Sea,CO2 Air,CO2 Total
0,2021-01-04 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,"FRANCE, LES ANGLES",42.579678,2.048950,765.728,0.0,0.0,0.0,20247100,963543-43,381.47,2.0,113.16800,8.318967,0.0,0.0,0.0,8.318967
1,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,"GERMANY, PEINE - WOLTORF",52.304594,10.312521,856.000,0.0,0.0,0.0,20203388,993081-81,24374.43,153.0,2530.32471,207.931963,0.0,0.0,0.0,207.931963
2,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,"FRANCE, BELLEVILLE",48.871184,2.386682,52.694,0.0,0.0,0.0,20203383,993081-81,20391.68,128.0,2116.87296,10.708464,0.0,0.0,0.0,10.708464
3,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,802077-77,408.27,300.0,5.70000,0.154030,0.0,0.0,0.0,0.154030
4,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,8022921-21,567.16,400.0,12.80000,0.345892,0.0,0.0,0.0,0.345892


#### Final mapping for visualization

In [10]:
df_agg['Delivery Mode'] = df_agg[MODES].astype(float).apply(
    lambda t: [mode if t[mode] > 0 else '-' for mode in MODES], axis = 1)
dict_map = dict(zip(df_agg['Delivery Mode'].astype(str).unique(),
                   [i.replace(", '-'",'').replace("'-'",'').replace("'",'') for i in df_agg['Delivery Mode'].astype(str).unique()]))
df_agg['Delivery Mode'] = df_agg['Delivery Mode'].astype(str).map(dict_map)
df_agg

,Date,Month-Year,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Location,GPS 1,GPS 2,Road,Rail,Sea,Air,Order Number,Item Code,Euros,Units,KG,CO2 Road,CO2 Rail,CO2 Sea,CO2 Air,CO2 Total,Delivery Mode
0,2021-01-04 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,"FRANCE, LES ANGLES",42.579678,2.048950,765.728,0.0,0.0,0.0,20247100,963543-43,381.47,2.0,113.16800,8.318967,0.0,0.0,0.0,8.318967,[Road]
1,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,"GERMANY, PEINE - WOLTORF",52.304594,10.312521,856.000,0.0,0.0,0.0,20203388,993081-81,24374.43,153.0,2530.32471,207.931963,0.0,0.0,0.0,207.931963,[Road]
2,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,"FRANCE, BELLEVILLE",48.871184,2.386682,52.694,0.0,0.0,0.0,20203383,993081-81,20391.68,128.0,2116.87296,10.708464,0.0,0.0,0.0,10.708464,[Road]
3,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,802077-77,408.27,300.0,5.70000,0.154030,0.0,0.0,0.0,0.154030,[Road]
4,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,8022921-21,567.16,400.0,12.80000,0.345892,0.0,0.0,0.0,0.345892,[Road]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5119,2021-10-28 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,21089497,897001-01,233.34,125.0,5.38000,0.145383,0.0,0.0,0.0,0.145383,[Road]
5120,2021-10-28 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,21089497,919289-89,132.24,80.0,1.36880,0.036989,0.0,0.0,0.0,0.036989,[Road]
5121,2021-10-28 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,21089497,942082-82,391.88,200.0,6.40000,0.172946,0.0,0.0,0.0,0.172946,[Road]
5122,2021-11-01 00:00:00.000,10-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,"GERMANY, PEINE - WOLTORF",52.304594,10.312521,856.000,0.0,0.0,0.0,21004473,982884-84,2980.00,2000.0,292.00000,23.995392,0.0,0.0,0.0,23.995392,[Road]


In [11]:
df_agg.to_csv('final_report.csv')